In [4]:
import pandas as pd
import os

# Load Excel file
file_path = "/Users/biekeleenknegt/Desktop/Supply Chain Management/SCM_GW/Real Demand.xlsx"
sheet_name = "Real Demand"

# Load data
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Rename 'season' to 'year'
df = df.rename(columns={'season': 'year'})

# Sort by year
df = df.sort_values(by='year')

# Get unique number of years per (product_id, size)
year_counts = df.groupby(['product_id', 'size'])['year'].nunique().reset_index()
year_counts = year_counts.rename(columns={'year': 'max_years'})

# Merge with original data
df = df.merge(year_counts, on=['product_id', 'size'], how='left')

# Create output folder if needed
output_folder = "/Users/biekeleenknegt/Documents/GitHub/SCM_GW/Forecasts_By_Window"
os.makedirs(output_folder, exist_ok=True)

# Track all unique window sizes
all_window_dfs = dict()

# Loop over each product-size group
for _, row in year_counts.iterrows():
    product = row['product_id']
    size = row['size']
    max_years = row['max_years']
    
    sub_df = df[(df['product_id'] == product) & (df['size'] == size)].copy()
    sub_df = sub_df.sort_values(by='year')
    
    for window in range(1, max_years + 1):
        sub_df[f'ma_{window}'] = sub_df['real demand'].rolling(window=window, min_periods=1).mean()
        forecast_row = sub_df.tail(1).copy()
        forecast_row['window'] = window
        forecast_row['forecast'] = forecast_row[f'ma_{window}']
        
        # Add to the right window's list
        if window not in all_window_dfs:
            all_window_dfs[window] = []
        all_window_dfs[window].append(forecast_row[['product_id', 'size', 'year', 'forecast']])

# Write one Excel file per window size
for window, dfs in all_window_dfs.items():
    result_df = pd.concat(dfs, ignore_index=True)
    output_path = os.path.join(output_folder, f"Forecast_Window_{window}.xlsx")
    result_df.to_excel(output_path, index=False)

print(f" Forecasts written to separate files in: {output_folder}")

 Forecasts written to separate files in: /Users/biekeleenknegt/Documents/GitHub/SCM_GW/Forecasts_By_Window
